In [1]:
import pandas as pd
import numpy as np
import names
import datetime
from IPython.display import display, HTML
from sqlalchemy import create_engine

display(HTML("<style>.container { width:100% !important; }</style>"))

# Creating a db

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///library.db

'Connected: @library.db'

In [4]:
%%sql 
SELECT name FROM sqlite_schema WHERE type='table'

 * sqlite:///library.db
Done.


name


In [5]:
%%sql
SELECT * FROM Event WHERE startTS BETWEEN "2022-01-01 00:00" AND "2024-01-01 00:00"

 * sqlite:///library.db
(sqlite3.OperationalError) no such table: Event
[SQL: SELECT * FROM Event WHERE startTS BETWEEN "2022-01-01 00:00" AND "2024-01-01 00:00"]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


# Creating tables

In [7]:
%%sql
CREATE TABLE User(
    userID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    age INTEGER NOT NULL,
    fines NUMERIC(5,2) DEFAULT 0,
    CHECK (age >= 7),
    CHECK (fines >= 0)
);

 * sqlite:///library.db
Done.


[]

In [8]:
%%sql
CREATE TABLE Librarian(
    librarianID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    salary NUMERIC(6,2) DEFAULT 0,
    department VARCHAR(20) NOT NULL,
    userID REFERENCES User(userID) UNIQUE DEFAULT NULL,
    CHECK (salary >= 0),
    CHECK ((department <> 'volunteer') OR (salary = 0)),
    CHECK (department IN ('admin', 'catalog', 'circulation', 'infotech', 'volunteer'))
);

 * sqlite:///library.db
Done.


[]

In [9]:
%%sql
CREATE TABLE BorrowedItem(
    userID REFERENCES User(userID) NOT NULL,
    libraryItemID REFERENCES LibraryItem(libraryItemID) NOT NULL,
    dueDate DATETIME DEFAULT (datetime('now', '+14 day')) NOT NULL,
    returnedDate DATETIME DEFAULT NULL,
    PRIMARY KEY (userID,libraryItemID,dueDate)
);

 * sqlite:///library.db
Done.


[]

In [10]:
%%sql
CREATE TABLE LibraryItem(
    libraryItemID INTEGER PRIMARY KEY,
    itemID REFERENCES Item(itemID) NOT NULL,
    toBeAdded BOOLEAN
);

 * sqlite:///library.db
Done.


[]

In [11]:
%%sql
CREATE TABLE Item(
    itemID INTEGER PRIMARY KEY,
    author VARCHAR(30) NOT NULL,
    itemName VARCHAR(30) NOT NULL,
    type VARCHAR(15) NOT NULL,
    CHECK (type IN ('movie', 'book', 'song', 'paper'))
);

 * sqlite:///library.db
Done.


[]

In [12]:
%%sql
CREATE TABLE Event(
    startTS DATETIME NOT NULL,
    endTS DATETIME NOT NULL,
    room VARCHAR(10) NOT NULL,
    eventName VARCHAR(30) NOT NULL,
    audience VARCHAR(20),
    maxRegistrations INTEGER DEFAULT 0,
    artist VARCHAR(30),
    itemID REFERENCES Item(itemID),
    PRIMARY KEY (startTS, room),
    CHECK (maxRegistrations >= 0)
);

 * sqlite:///library.db
Done.


[]

In [13]:
%%sql
CREATE TABLE EventRegistration(
    startTS REFERENCES Event(startTS) NOT NULL,
    room REFERENCES Event(room) NOT NULL,
    userID REFERENCES User(userID) NOT NULL,
    PRIMARY KEY (startTS, room, userID)
);

 * sqlite:///library.db
Done.


[]

# ASSERTIONS AND TRIGGERS

## List of Assertions

 - Maximum possible users <= some constant
 - Maximum employee salary <= budget

## List of Triggers

 - Users with fines cannot borrow items
 - Users cannot take items that are already checked out
 - Users cannot return items that have already been returned
 - Users cannot register for events at max capacity
 - Users cannot register multiple times for the same event
 - Users cannot register for past events

In [21]:
%%sql
CREATE TRIGGER prevent_multiple_returns
BEFORE UPDATE OF returnedDate ON BorrowedItem
FOR EACH ROW
WHEN NEW.returnedDate IS NOT NULL AND OLD.returnedDate IS NOT NULL
BEGIN
    SELECT RAISE(ABORT, 'The item has already been returned.');
END;

 * sqlite:///library.db
Done.


[]

In [22]:
%%sql
CREATE TRIGGER prevent_max_registrations
BEFORE INSERT ON EventRegistration
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM EventRegistration WHERE startTS = NEW.startTS AND room = NEW.room
) >= (
    SELECT maxRegistrations FROM Event WHERE startTS = NEW.startTS AND room = NEW.room
)
BEGIN
    SELECT RAISE(ABORT, 'Event is at max capacity.');
END;

 * sqlite:///library.db
Done.


[]

In [23]:
%%sql
CREATE TRIGGER prevent_duplicate_event_registration
BEFORE INSERT ON EventRegistration
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM EventRegistration
    WHERE startTS = NEW.startTS AND room = NEW.room AND userID = NEW.userID
) > 0
BEGIN
    SELECT RAISE(ABORT, 'The user has already registered for this event.');
END;

 * sqlite:///library.db
Done.


[]

In [24]:
%%sql
CREATE TRIGGER prevent_past_event_registration
BEFORE INSERT ON EventRegistration
FOR EACH ROW
WHEN (
    SELECT datetime(startTS) FROM Event WHERE startTS = NEW.startTS AND room = NEW.room
) < datetime('now')
BEGIN
    SELECT RAISE(ABORT, 'Cannot register for past events.');
END;

 * sqlite:///library.db
Done.


[]

In [25]:
%%sql
CREATE TRIGGER prevent_borrow_unavailable_item
BEFORE INSERT ON BorrowedItem
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM BorrowedItem 
    WHERE libraryItemID = NEW.libraryItemID 
    AND returnedDate IS NULL
) > 0
BEGIN
    SELECT RAISE(ABORT, 'The item is not available for borrowing.');
END;

 * sqlite:///library.db
Done.


[]

In [26]:
%%sql
CREATE TRIGGER prevent_borrow_unavailable_item_tba
BEFORE INSERT ON BorrowedItem
FOR EACH ROW
WHEN (
    SELECT toBeAdded FROM LibraryItem WHERE libraryItemID = NEW.libraryItemID
) = 1
BEGIN
    SELECT RAISE(ABORT, 'The item is not available for borrowing.');
END;

 * sqlite:///library.db
Done.


[]

In [27]:
%%sql
CREATE TRIGGER add_fine_past_due
AFTER UPDATE ON BorrowedItem
FOR EACH ROW
WHEN (
    datetime('now') > NEW.dueDate
    AND NEW.returnedDate IS NULL
    AND NEW.dueDate IS NOT NULL
)
BEGIN
    UPDATE User
    SET fines = fines + 3.00
    WHERE userID = NEW.userID AND fines = 0.00;
END;

 * sqlite:///library.db
Done.


[]

#### ignore

In [29]:
%%sql
CREATE TRIGGER prevent_excessive_borrowing
BEFORE INSERT ON BorrowedItem
FOR EACH ROW
WHEN (
    SELECT COUNT(*) FROM BorrowedItem WHERE userID = NEW.userID AND returnedDate IS NULL
) >= 3
BEGIN
    SELECT RAISE(ABORT, 'Cannot borrow more than three items.');
END;

 * sqlite:///library.db
Done.


[]

In [29]:
%%sql
SELECT * FROM sqlite_master WHERE type = 'trigger'; 

 * sqlite:///library.db
Done.


type,name,tbl_name,rootpage,sql
trigger,prevent_multiple_returns,BorrowedItem,0,"CREATE TRIGGER prevent_multiple_returnsBEFORE UPDATE OF returnedDate ON BorrowedItemFOR EACH ROWWHEN NEW.returnedDate IS NOT NULL AND OLD.returnedDate IS NOT NULLBEGIN SELECT RAISE(ABORT, 'The item has already been returned.');END"
trigger,prevent_max_registrations,EventRegistration,0,"CREATE TRIGGER prevent_max_registrationsBEFORE INSERT ON EventRegistrationFOR EACH ROWWHEN ( SELECT COUNT(*) FROM EventRegistration WHERE startTS = NEW.startTS AND room = NEW.room) >= ( SELECT maxRegistrations FROM Event WHERE startTS = NEW.startTS AND room = NEW.room)BEGIN SELECT RAISE(ABORT, 'Event is at max capacity.');END"
trigger,prevent_duplicate_event_registration,EventRegistration,0,"CREATE TRIGGER prevent_duplicate_event_registrationBEFORE INSERT ON EventRegistrationFOR EACH ROWWHEN ( SELECT COUNT(*) FROM EventRegistration WHERE startTS = NEW.startTS AND room = NEW.room AND userID = NEW.userID) > 0BEGIN SELECT RAISE(ABORT, 'The user has already registered for this event.');END"
trigger,prevent_past_event_registration,EventRegistration,0,"CREATE TRIGGER prevent_past_event_registrationBEFORE INSERT ON EventRegistrationFOR EACH ROWWHEN ( SELECT datetime(startTS) FROM Event WHERE startTS = NEW.startTS AND room = NEW.room) < datetime('now')BEGIN SELECT RAISE(ABORT, 'Cannot register for past events.');END"
trigger,prevent_borrow_unavailable_item,BorrowedItem,0,"CREATE TRIGGER prevent_borrow_unavailable_itemBEFORE INSERT ON BorrowedItemFOR EACH ROWWHEN ( SELECT COUNT(*) FROM BorrowedItem WHERE libraryItemID = NEW.libraryItemID AND returnedDate IS NULL) > 0BEGIN SELECT RAISE(ABORT, 'The item is not available for borrowing.');END"
trigger,prevent_borrow_unavailable_item_tba,BorrowedItem,0,"CREATE TRIGGER prevent_borrow_unavailable_item_tbaBEFORE INSERT ON BorrowedItemFOR EACH ROWWHEN ( SELECT toBeAdded FROM LibraryItem WHERE libraryItemID = NEW.libraryItemID) = 1BEGIN SELECT RAISE(ABORT, 'The item is not available for borrowing.');END"
trigger,add_fine_past_due,BorrowedItem,0,CREATE TRIGGER add_fine_past_dueAFTER UPDATE ON BorrowedItemFOR EACH ROWWHEN ( datetime('now') > NEW.dueDate AND NEW.returnedDate IS NULL AND NEW.dueDate IS NOT NULL)BEGIN UPDATE User SET fines = fines + 3.00 WHERE userID = NEW.userID AND fines = 0.00;END


# INDEXING

In [13]:
%%sql
SELECT * FROM sqlite_master WHERE type = 'index'; 

 * sqlite:///library.db
Done.


type,name,tbl_name,rootpage,sql
index,sqlite_autoindex_BorrowedItem_1,BorrowedItem,5,None
index,sqlite_autoindex_Event_1,Event,9,None
index,sqlite_autoindex_EventRegistration_1,EventRegistration,11,None


In [20]:
%%sql
--CREATE INDEX pk_user_id ON User(userID);
--CREATE INDEX pk_librarian_id ON Librarian(librarianID);
--CREATE INDEX pk_libraryitem_id ON LibraryItem(libraryItemID);
--CREATE INDEX pk_item_id ON Item(itemID);
CREATE INDEX ck_libraryitem_id ON LibraryItem(itemID);
CREATE INDEX ck_event_room ON Event(room);
CREATE INDEX ck_eventregistration_userid ON EventRegistration(userID);


 * sqlite:///library.db
Done.
Done.
Done.


[]

# Populating tables

## Table: Item

### Books

In [22]:
books = pd.read_csv('..//data//books2.csv').iloc[:10]
books = books[['author', 'title']]
books.rename(columns={'title':'itemName'}, inplace=True)
books.loc[:,'type'] = 'book'
books.drop_duplicates(subset=['itemName'], inplace=True)
books.head(5)

,author,itemName,type
0,Suzanne Collins,The Hunger Games,book
1,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter and the Order of the Phoenix,book
2,Harper Lee,To Kill a Mockingbird,book
3,"Jane Austen, Anna Quindlen (Introduction)",Pride and Prejudice,book
4,Stephenie Meyer,Twilight,book


### Movies

In [23]:
movies = pd.read_csv('..//data/movies.csv')[:5]
movies.rename(columns={'title':'itemName'}, inplace=True)
for row in range(0,len(movies)):
    movies.loc[row,'itemName'] = movies.iloc[row]['itemName'][:-7]
movies.loc[:, 'type'] = 'movie'
movies.loc[:, 'author'] = 'nan'
movies.drop(columns=['movieId', 'genres'], inplace=True)
movies.head(5)

,itemName,type,author
0,Toy Story,movie,nan
1,Jumanji,movie,nan
2,Grumpier Old Men,movie,nan
3,Waiting to Exhale,movie,nan
4,Father of the Bride Part II,movie,nan


### Songs

In [24]:
songs_full = pd.read_csv('..//data/songs.csv')
songs_full.rename(columns={'song':'itemName','artist':'author'}, inplace=True)
songs = pd.DataFrame()
for row in range(0,5):
    random_song = songs_full.sample()
    songs = pd.concat([songs,random_song])
songs.loc[:,'type'] = 'song'
songs.drop(columns=['link', 'text'], inplace=True)
songs.head(5)

,author,itemName,type
56959,XTC,Funk Pop A Roll,song
30238,Doobie Brothers,Daughters Of The Sea,song
37033,INXS,Black And White,song
10586,King Crimson,Exiles,song
56802,Wyclef Jean,Low Income,song


### Papers

In [25]:
paper1 = pd.DataFrame(pd.Series({'author':'Abigail See', 'itemName':'A Discourse-Aware Attention Model for Abstractive Summarization of Long Documents', 'type':'paper'})).transpose()
paper2 = pd.DataFrame(pd.Series({'author':'Martin Ester', 'itemName':'A Density-Based Algorithm for Discovering Clusters in Large Spatial Databases with Noise ', 'type':'paper'})).transpose()
papers = pd.concat([paper1, paper2])
papers

,author,itemName,type
0,Abigail See,A Discourse-Aware Attention Model for Abstract...,paper
0,Martin Ester,A Density-Based Algorithm for Discovering Clus...,paper


In [26]:
Item = pd.concat([books, movies, songs, papers])
Item.reset_index(drop=True, inplace=True)
Item.index.names = ['itemID']

In [27]:
for row in range(0,len(Item)):
    itemName = Item.iloc[row]['author']
    comma_index = itemName.find(',')
    if (comma_index>1):
        author_solo = itemName[0:comma_index]
        Item.loc[row,'author'] = author_solo


In [28]:
Item

,author,itemName,type
itemID,,,
0,Suzanne Collins,The Hunger Games,book
1,J.K. Rowling,Harry Potter and the Order of the Phoenix,book
2,Harper Lee,To Kill a Mockingbird,book
3,Jane Austen,Pride and Prejudice,book
4,Stephenie Meyer,Twilight,book
5,Markus Zusak (Goodreads Author),The Book Thief,book
6,George Orwell,Animal Farm,book
7,C.S. Lewis,The Chronicles of Narnia,book
8,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,book


In [29]:
Item.to_csv('..//data//Item.csv')

## LibraryItem

In [30]:
num_copy = 8
LibraryItem = Item.copy()
for i in range(num_copy):
    item_copy = Item.sample()
    LibraryItem = pd.concat([LibraryItem, item_copy])


In [31]:
LibraryItem.reset_index(inplace=True)
LibraryItem.index.names = ['libraryItemID']
LibraryItem.drop(columns=['author','itemName','type'], inplace=True)
LibraryItem['toBeAdded'] = None

In [32]:
LibraryItem.loc[12,'toBeAdded'] = True

In [33]:
LibraryItem.to_csv('..//data//LibraryItem.csv')

## Librarian

In [35]:
numLibrarians = 10
departments = ['admin', 'catalog', 'circulation', 'infotech']
Librarian = pd.DataFrame()
for librarianID in range(1,numLibrarians+1):
    firstName = names.get_first_name()
    lastName = names.get_last_name()
    department = np.random.choice(departments)

    salary = np.random.randint(40000,90000)
    salary = round(salary, -3)
        
    librarian = pd.DataFrame(pd.Series({'firstName':firstName, 'lastName':lastName, 'department':department, 'salary':salary})).transpose()
    Librarian = pd.concat([Librarian, librarian])
    

librarian = pd.DataFrame(pd.Series({'firstName':'Joe', 'lastName':'Black', 'department':'volunteer', 'salary':0})).transpose()
Librarian = pd.concat([Librarian, librarian])
    
Librarian.dropna(inplace=True)


In [36]:
Librarian.reset_index(drop=True, inplace=True)
Librarian.index.names = ['librarianID']
Librarian

,firstName,lastName,department,salary
librarianID,,,,
0,Barbara,Fortner,infotech,46000
1,Eugenia,King,admin,61000
2,Joseph,Taylor,circulation,52000
3,Robert,Grays,admin,48000
4,Lucy,Johnson,catalog,71000
5,Amy,Espada,catalog,52000
6,Joseph,Barrier,admin,90000
7,Linda,Lamb,admin,74000
8,Luella,Holiday,admin,76000


In [37]:
Librarian.to_csv('..//data//Librarian.csv')

## User

In [38]:
numUsers = 30
User = pd.DataFrame()
for librarianID in range(numUsers):
    firstName = names.get_first_name()
    lastName = names.get_last_name()
    age = np.random.randint(7,75)
        
    user = pd.DataFrame(pd.Series({'firstName':firstName, 'lastName':lastName, 'age':age, 'fines': 0.00})).transpose()
    User = pd.concat([User, user])
    

In [39]:
User.reset_index(drop=True, inplace=True)
User.index.names = ['userID']
User.loc[14,'fines'] = 3.00
User.loc[28,'fines'] = 3.00
User

,firstName,lastName,age,fines
userID,,,,
0,Arthur,Thompson,64,0.0
1,Della,Mcentire,53,0.0
2,Danny,Garcia,43,0.0
3,Carolina,Smith,27,0.0
4,Janet,Conzemius,11,0.0
5,Travis,Rodney,25,0.0
6,Ira,Smith,9,0.0
7,Christy,Sweitzer,69,0.0
8,Wendell,Smith,42,0.0


In [40]:
User.to_csv('..//data//User.csv')

## BorrowedItem

In [28]:
User = pd.read_csv('..//data//User.csv')
LibraryItem = pd.read_csv('..//data//LibraryItem.csv')

In [42]:
numBorrowedItem = 10
BorrowedItem = pd.DataFrame()
itemList = []

for i in range(numBorrowedItem):
    userID = np.random.choice(User.reset_index()['userID'])
    dueDate = datetime.datetime(2023,7,25,17) + datetime.timedelta(days=i)
    returnedDay = np.random.randint(0,10)
    returnedDate = dueDate - datetime.timedelta(days=returnedDay)
    libraryItemID = np.random.choice(LibraryItem.reset_index()['libraryItemID'])
    while(libraryItemID in itemList):
        libraryItemID = np.random.choice(LibraryItem.reset_index()['libraryItemID'])
    
    bi = pd.DataFrame(pd.Series({'userID':userID, 'libraryItemID':libraryItemID, 'dueDate':dueDate, 'returnDate':returnedDate})).transpose()
    BorrowedItem = pd.concat([BorrowedItem, bi])
    


In [43]:
BorrowedItem

,userID,libraryItemID,dueDate,returnDate
0,9,20,2023-07-25 17:00:00,2023-07-22 17:00:00
0,12,13,2023-07-26 17:00:00,2023-07-25 17:00:00
0,3,17,2023-07-27 17:00:00,2023-07-24 17:00:00
0,23,11,2023-07-28 17:00:00,2023-07-27 17:00:00
0,1,14,2023-07-29 17:00:00,2023-07-25 17:00:00
0,25,4,2023-07-30 17:00:00,2023-07-21 17:00:00
0,4,21,2023-07-31 17:00:00,2023-07-28 17:00:00
0,11,18,2023-08-01 17:00:00,2023-07-30 17:00:00
0,27,14,2023-08-02 17:00:00,2023-07-28 17:00:00
0,28,10,2023-08-03 17:00:00,2023-07-27 17:00:00


In [44]:
BorrowedItem.to_csv('..//data/BorrowedItem.csv')

## Event and Registration

In [14]:
Item = pd.read_csv('..//data/Item.csv')
LibraryItem = pd.read_csv('..//data/LibraryItem.csv')
Librarian = pd.read_csv('..//data/Librarian.csv')
User = pd.read_csv('..//data/User.csv')
Event = pd.read_csv('..//data/Event.csv')
EventRegistration = pd.read_csv('..//data/EventRegistration.csv')
BorrowedItem  = pd.read_csv('..//data/BorrowedItem.csv')

In [15]:
BorrowedItem

,userID,libraryItemID,dueDate,returnedDate
0,9,20,2023-07-25 17:00,2023-07-22 17:00
1,12,13,2023-07-26 17:00,2023-07-25 17:00
2,3,17,2023-07-27 17:00,2023-07-24 17:00
3,23,11,2023-07-28 17:00,2023-07-27 17:00
4,1,14,2023-07-29 17:00,2023-07-25 17:00
5,25,4,2023-07-30 17:00,2023-07-21 17:00
6,4,21,2023-07-31 17:00,NaN
7,11,18,2023-08-01 17:00,2023-07-30 17:00
8,27,14,2023-08-07 17:00,NaN
9,28,10,2023-08-07 17:00,NaN


In [16]:
Event.sort_values(by=['startTS','room'], inplace=True),
EventRegistration.sort_values(by=['startTS','room', 'userID'], inplace=True)

In [17]:
engine = create_engine('sqlite:///library.db', echo=False)
Item.to_sql('Item', con=engine, if_exists='append', index=False)
LibraryItem.to_sql('LibraryItem', con=engine, if_exists='append', index=False)
Librarian.to_sql('Librarian', con=engine, if_exists='append', index=False)
User.to_sql('User', con=engine, if_exists='append', index=False)
BorrowedItem.to_sql('BorrowedItem', con=engine, if_exists='append', index=False)
Event.to_sql('Event', con=engine, if_exists='append', index=False)
EventRegistration.to_sql('EventRegistration', con=engine, if_exists='append', index=False)


9

In [19]:
%%sql
SELECT * FROM BorrowedItem

 * sqlite:///library.db
Done.


userID,libraryItemID,dueDate,returnedDate
9,20,2023-07-25 17:00,2023-07-22 17:00
12,13,2023-07-26 17:00,2023-07-25 17:00
3,17,2023-07-27 17:00,2023-07-24 17:00
23,11,2023-07-28 17:00,2023-07-27 17:00
1,14,2023-07-29 17:00,2023-07-25 17:00
25,4,2023-07-30 17:00,2023-07-21 17:00
4,21,2023-07-31 17:00,None
11,18,2023-08-01 17:00,2023-07-30 17:00
27,14,2023-08-07 17:00,None
28,10,2023-08-07 17:00,None


In [158]:
%%sql
SELECT * FROM BorrowedItem;

 * sqlite:///library.db
Done.


userID,libraryItemID,dueDate,returnedDate
9,20,2023-07-25 17:00,2023-07-22 17:00:00
12,13,2023-07-26 17:00,2023-07-25 17:00:00
3,17,2023-07-27 17:00,2023-07-24 17:00:00
23,11,2023-07-28 17:00,2023-07-27 17:00:00
1,14,2023-07-29 17:00,2023-07-25 17:00:00
25,4,2023-07-30 17:00,2023-07-21 17:00:00
4,21,2023-07-31 17:00,None
11,18,2023-08-01 17:00,2023-07-30 17:00:00
27,14,2023-08-07 17:00,None
28,10,2023-08-07 17:00,None


In [150]:
%%sql
INSERT INTO BorrowedItem(userID, libraryItemID, dueDate, returnedDate)
VALUES(9,20,'2023-07-25 17:00','2023-07-22 17:00:00'),
    (12,13,'2023-07-26 17:00','2023-07-25 17:00:00'),
    (3,17,'2023-07-27 17:00','2023-07-24 17:00:00'),
    (23,11,'2023-07-28 17:00','2023-07-27 17:00:00'),
    (1,14,'2023-07-29 17:00','2023-07-25 17:00:00'),
    (25,4,'2023-07-30 17:00','2023-07-21 17:00:00'),
    (4,21,'2023-07-31 17:00',NULL),
    (11,18,'2023-08-01 17:00','2023-07-30 17:00:00'),
    (27,14,'2023-08-07 17:00',NULL),
    (28,10,'2023-08-07 17:00',NULL);


 * sqlite:///library.db
10 rows affected.


[]

## Table: User

In [138]:
%%sql
DROP TABLE mytable

 * sqlite:///library.db
Done.


[]

In [128]:
%%sql

DROP TABLE Item;
DROP TABLE User;
DROP TABLE Librarian;
DROP TABLE LibraryItem;
DROP TABLE BorrowedItem;
DROP TABLE Event;
DROP TABLE EventRegistration;

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [52]:
%%sql

DROP TRIGGER prevent_past_event_registration;

 * sqlite:///library.db
Done.


[]

In [37]:
queryList = []

In [39]:
queryList.append("s")
queryList.append()

In [40]:
queryList

['s']

In [50]:
import sqlite3
from os import path

conn = sqlite3.connect("library.db")
eventQuery = "SELECT * FROM Event"
# Get rows
with conn:
    cur = conn.cursor()
    try:
        cur.execute(eventQuery)
    except:
        print("Unable to execute query")

In [51]:
rows = cur.fetchall()

In [57]:
rows[0][2]

'A1'